## Import libs

In [1]:
import tensorflow as tf
from pathlib import Path
from util.dataset import Dataset
from util import read_data as rd
from tensorflow.keras import layers
from madras_laftr import trainer, tester
from madras_laftr.models import DemParGan
from util.results import ResultLogger

## Preliminaries

In [2]:
batch_size = 64
activation = 'leaky_relu'
learning_rate = 0.001
optmizer = 'adam'
class_coeff = 1.0
fair_coeff = 1.0
recon_coeff = 0.0
model_type = 'DemParGan'

## Load data

In [3]:
data_name = 'adult'

data_info = rd.return_data_info(data_name)
npzfile = rd.return_npz(data_name)

data = Dataset(npzfile=npzfile, name=data_name, a0_name=data_info['a0_name'], a1_name=data_info['a1_name'], 
                use_a=data_info['use_a'], seed=data_info['seed'], batch_size=batch_size)

shapes = data.get_shapes()
print(shapes)

y shape (30162, 2)
changing shape
((24128, 113), (24128, 1), (24128, 1), (15040, 113), (15040, 1), (15040, 1), (6016, 113), (6016, 1), (6016, 1))


In [4]:
data.x_train[0]

array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  9.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0., 50.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.])

In [5]:
if 'Weighted' in model_type:
        A_weights = [1. / x for x in data.get_A_proportions()]
        Y_weights = [1. / x for x in data.get_Y_proportions()]
        AY_weights = [[1. / x for x in L] for L in data.get_AY_proportions()]

        fair_metric = 'DP' #'EODD' #'EOPP'
        if fair_metric == 'EOPP':
                AY_weights[0][1] = 0. #AY_weights[0][1]
                AY_weights[1][1] = 0. #AY_weights[1][1]

## Model

In [6]:
resdirname = Path(r'/home/luiz/ufpb/mestrado/code/falsb/data/')
resdirname = resdirname/data_name/'post_prep'

model = DemParGan()

with tf.compat.v1.Session() as sess:
    reslogger = ResultLogger(dname=resdirname)

    #create Trainer
    trainer = Trainer(model, data, sess=sess, expdir=resdirname, logs_path=logdirname,
                          **args['optim'], **args['train'])

    # training
    trainer.train(**args['train'])

    # test the trained model
    tester = Tester(model, data, sess, reslogger)
    tester.evaluate(args['train']['batch_size'])

# flush
tf.reset_default_graph()

AttributeError: module 'tensorflow' has no attribute 'placeholder'